# Calculadora de avance

In [151]:
import pandas as pd
import os
import pickle
import json
import xlsxwriter
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import re
from typing import Dict, List, Any
import itertools

from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from google.oauth2 import service_account
from google.cloud import firestore

import firebase_admin
import excel2img
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

## Datos del proyecto en análisis

In [152]:
proyecto="sibayo"
mes=4
anio=2025

## Datos de Firestore

In [153]:
## Datos de # 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError("No se encontró ningún archivo .env en este directorio ni en sus padres.")
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [154]:
db=firestore.Client(credentials=credentials, project=credentials.project_id)

In [155]:
# 4. Define la ruta a tu documento anidado
colec_raiz = "rutinarios"
doc_proyecto = proyecto     # puede ser tu variable proyecto
colec_valoriz = "valorizaciones"
id_valoriz = str(mes)

doc_ref = (
    db
    .collection(colec_raiz)
    .document(doc_proyecto)
    .collection(colec_valoriz)
    .document(id_valoriz)
)

data_mantenimiento_snapshot=doc_ref.get()

if not data_mantenimiento_snapshot.exists:
    print(f"El documento {doc_ref.path} no existe.")

data_mantenimiento= data_mantenimiento_snapshot.to_dict()
print(data_mantenimiento)

{'cargas_trabajo_contratista': {'MR101': 0.86, 'MR202': 6.0, 'MR301': 3600.0, 'MR601': 64.5, 'MR201': 2720.0, 'MR401': 5}}


In [156]:
cargas_trabajo_contratista_presente_mes=data_mantenimiento['cargas_trabajo_contratista']

In [157]:
data_mantenimiento=db.collection("rutinarios").document(proyecto).get()
if data_mantenimiento.exists:
    print(data_mantenimiento.to_dict())
else:
    print("No existe el documento.")

{'otros': {'tipo_superficie': 'Asfaltado', 'estado_conservacion_via': 'regular'}, 'other': {'tipo_superficie': 'Asfaltado', 'estado_conservacion_via': 'regular'}, 'expediente': {'codigo_ruta': 'AR-683', 'monto_total': 206413.0, 'coordenadas': {'fin': {'hemisferio': 'S', 'y': 8270075.15, 'progresiva': 32252.0, 'zona_letra': None, 'x': 220383.97, 'datum': 'WGS84', 'altitud': 3629, 'zona': None}, 'inicio': {'hemisferio': 'S', 'y': 8281146.79, 'progresiva': 0, 'zona_letra': None, 'x': 226316.6, 'datum': 'WGS84', 'altitud': 3820, 'zona': None}}, 'nombre': 'MANTENIMIENTO VIAL RUTINARIO CAMINO VECINAL EMP. AR-111 (NUEVO SIBAYO) - TUTI - EMP.AR-681 (DV. CHIVAY), DISTRITOS DE CHIVAY, TUTI Y SIBAYO, PROVINCIA DE CAYLLOMA, DEPARTAMENTO DE AREQUIPA', 'longitud': 32252.0}, 'datos_generales': {'distritos': ['Chivay', 'Tuti', 'Sibayo'], 'region': 'Arequipa', 'provincia': 'Caylloma'}, 'contrato': {'jefe_mantenimiento': {'titulo': 'Ingeniero', 'nombre': 'Genaro', 'dni': 0, 'apellido': 'Tinta Cáceres'},

In [158]:
progresiva_inicio=data_mantenimiento.to_dict()["expediente"]["coordenadas"]["inicio"]["progresiva"]
progresiva_fin=data_mantenimiento.to_dict()["expediente"]["coordenadas"]["fin"]["progresiva"]

print(progresiva_inicio)
print(progresiva_fin)

0
32252.0


## Cargando cargas de trabajo programadas

In [159]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cargas_trabajo.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    df_cargas_trabajo_programadas = pickle.load(f)

df_cargas_trabajo_programadas.tail(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR111,92.230488,202.907073,184.460976,202.907073,202.907073,184.460976,202.907073,184.460976,55.338293,1512.58
MR112,52.500000,87.500000,87.500000,87.500000,87.500000,87.500000,87.500000,87.500000,35.000000,700.00
MR201,2205.681818,4411.363636,3676.136364,4411.363636,4411.363636,3676.136364,4411.363636,3676.136364,1470.454545,32350.00
MR202,3.059118,5.098529,4.078824,4.078824,4.078824,4.078824,4.078824,4.078824,2.039412,34.67
MR203,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR206,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR301,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,0.000000,15999.00


In [160]:
#eliminando el total
df_cargas_trabajo_programadas = df_cargas_trabajo_programadas.iloc[:-1]

In [161]:
# Concatenar la ruta completa al archivo .pkl
ruta_cronograma_anual = os.path.join("data",proyecto, f"{proyecto}_cronograma_anual.pkl")

# Leer el archivo pickle
with open(ruta_cronograma_anual, "rb") as f:
    df_cronograma_anual = pickle.load(f)

df_cronograma_anual.tail(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
MR111,5,11,10,11,11,10,11,10,3,82
MR112,3,5,5,5,5,5,5,5,2,40
MR201,3,6,5,6,6,5,6,5,2,44
MR202,3,5,4,4,4,4,4,4,2,34
MR203,0,0,0,0,0,0,0,0,0,0
MR204,0,0,0,0,0,0,0,0,0,0
MR205,0,0,0,0,0,0,0,0,0,0
MR206,0,0,0,0,0,0,0,0,0,0
MR301,1,1,1,1,1,1,1,1,0,8
MR401,0,0,1,0,0,1,1,1,0,4


## Cargando actividades

In [162]:
ruta_actividades= os.path.join("data", "general_data", "actividades.json")
with open(ruta_actividades, 'r', encoding='utf-8') as archivo:
    actividades = json.load(archivo)
# Ahora 'datos' es un diccionario de Python
print(actividades)

[{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 0, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 0, 'unidad': 'm3'}}]}}, {'key': 'MR200', 'value': {'label': 'Limpieza de obras de arte', 'value': [{'key': 'MR201', 'value': {'label': 'Limpieza de cunetas', 'carga_trabajo': 1440, 'unidad': 'm'}}, {'key': 'MR202', 'value': {'label': 'Limpieza de alcantarillas', 'carga_trabajo': 0, 'unidad': 'unidad'}}, {'key': 'MR203', 'value': {'label': 'Limpieza de badén', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR204', 'value': {'label': 'Limpieza de zanjas de coronación', 'carga_trabajo': 0, 'unidad': 'm'}}, {'key': 'MR205', 'value': {'label': 'Limpieza de pontones', 'carga_tra

## Cargando cargas de trabajo del contratista

In [163]:
# ruta_cargas_trabajo= os.path.join("data", proyecto,str(mes), f"cargas_trabajo.json")
# with open(ruta_cargas_trabajo, 'r', encoding='utf-8') as archivo:
#     cargas_trabajo_contratista_presente_mes = json.load(archivo)
# # Ahora 'cargas_trabajo_contratista' es un diccionario de Python
# print(cargas_trabajo_contratista_presente_mes)

## Funciones utiles

In [164]:
def calcular_porcentaje(cargas_mes, cargas_totales, carga_por_dia):
    """
    Devuelve el porcentaje del proyecto programado para este mes,
    ponderado por la carga de trabajo diaria de cada actividad.
    """
    total_dias_mes = 0.0
    total_dias_proyecto = 0.0

    for actividad, dias_diarios in carga_por_dia.items():
        # Ignoramos actividades sin capacidad diaria definida o totales cero
        if dias_diarios <= 0:
            continue

        carga_mes = cargas_mes.get(actividad, 0.0)
        carga_total = cargas_totales.get(actividad, 0.0)

        # Convertimos cargas a días
        total_dias_mes += carga_mes / dias_diarios
        total_dias_proyecto += carga_total / dias_diarios
        # print(
        #     {
        #         "actividad": actividad,
        #         "total_dias_mes": total_dias_mes,
        #         "total_dias_proyecto": total_dias_proyecto,
        #     }
        # )

    if total_dias_proyecto == 0:
        return 0.0

    return total_dias_mes / total_dias_proyecto

## Calculo

In [165]:
actividades=df_cargas_trabajo_programadas.index.to_list()
print(actividades)

['MR101', 'MR102', 'MR103', 'MR104', 'MR111', 'MR112', 'MR201', 'MR202', 'MR203', 'MR204', 'MR205', 'MR206', 'MR301', 'MR401', 'MR501', 'MR601', 'MR701', 'MR702']


### Avance programado vigente en el mes

In [166]:
# Buscamos la columna que termina en "-04"
col = [c for c in df_cargas_trabajo_programadas.columns if c.endswith(f"-{mes:02d}")][0]

# Extraemos la Serie y la pasamos a dict
cargas_trabajo_programdas_presente_mes = df_cargas_trabajo_programadas[col].to_dict()

print(cargas_trabajo_programdas_presente_mes)
# {'MR101': 1.540444, 'MR102': 153.14833, ... }

{'MR101': 0.8566666666666666, 'MR102': 0.0, 'MR103': 0.0, 'MR104': 0.0, 'MR111': 92.23048780487804, 'MR112': 52.5, 'MR201': 2205.681818181818, 'MR202': 3.059117647058824, 'MR203': 0.0, 'MR204': 0.0, 'MR205': 0.0, 'MR206': 0.0, 'MR301': 1999.875, 'MR401': 0.0, 'MR501': 0.0, 'MR601': 0.0, 'MR701': 0.0, 'MR702': 0.0}


In [167]:
# Opción A: usando df.columns
ultima_col = df_cargas_trabajo_programadas.columns[-1]
cargas_trabajo_programadas_totales = df_cargas_trabajo_programadas[ultima_col].to_dict()
print(cargas_trabajo_programadas_totales)

{'MR101': 10.28, 'MR102': 0.0, 'MR103': 53.33, 'MR104': 85.33, 'MR111': 1512.58, 'MR112': 700.0, 'MR201': 32350.0, 'MR202': 34.67, 'MR203': 0.0, 'MR204': 0.0, 'MR205': 0.0, 'MR206': 0.0, 'MR301': 15999.0, 'MR401': 64.67, 'MR501': 0.0, 'MR601': 258.02, 'MR701': 0.0, 'MR702': 0.0}


In [168]:
dias_totales_por_actividad=df_cronograma_anual[ultima_col].to_dict()
print(dias_totales_por_actividad)

{'MR101': 12, 'MR102': 0, 'MR103': 4, 'MR104': 6, 'MR111': 82, 'MR112': 40, 'MR201': 44, 'MR202': 34, 'MR203': 0, 'MR204': 0, 'MR205': 0, 'MR206': 0, 'MR301': 8, 'MR401': 4, 'MR501': 0, 'MR601': 6, 'MR701': 0, 'MR702': 0, 'TOTAL': 240}


In [169]:
carga_por_dia = {
    k: (
        (cargas_trabajo_programadas_totales.get(k, 0) / dias_totales_por_actividad[k])
        if dias_totales_por_actividad[k] != 0
        else 0
    )
    for k in dias_totales_por_actividad
}

print(carga_por_dia)

{'MR101': 0.8566666666666666, 'MR102': 0, 'MR103': 13.3325, 'MR104': 14.221666666666666, 'MR111': 18.44609756097561, 'MR112': 17.5, 'MR201': 735.2272727272727, 'MR202': 1.0197058823529412, 'MR203': 0, 'MR204': 0, 'MR205': 0, 'MR206': 0, 'MR301': 1999.875, 'MR401': 16.1675, 'MR501': 0, 'MR601': 43.00333333333333, 'MR701': 0, 'MR702': 0, 'TOTAL': 0.0}


In [170]:
porcentajes_programado_vigente_mes = {
    k: (cargas_trabajo_programdas_presente_mes.get(k, 0) / cargas_trabajo_programadas_totales[k]
        if cargas_trabajo_programadas_totales[k] != 0 else 0)
    for k in cargas_trabajo_programadas_totales
}
print(porcentajes_programado_vigente_mes)

{'MR101': 0.08333333333333333, 'MR102': 0, 'MR103': 0.0, 'MR104': 0.0, 'MR111': 0.06097560975609756, 'MR112': 0.075, 'MR201': 0.06818181818181818, 'MR202': 0.08823529411764706, 'MR203': 0, 'MR204': 0, 'MR205': 0, 'MR206': 0, 'MR301': 0.125, 'MR401': 0.0, 'MR501': 0, 'MR601': 0.0, 'MR701': 0, 'MR702': 0}


In [171]:
porcentaje_programado_vigente_mes=calcular_porcentaje(cargas_mes=cargas_trabajo_programdas_presente_mes,cargas_totales=cargas_trabajo_programadas_totales,carga_por_dia=carga_por_dia)
print(porcentaje_programado_vigente_mes)

0.06666666666666667


#### Como no calcular

In [172]:
"""
En este script calculamos el avance mensual de dos formas:

1. Avance ponderado por carga de trabajo:
   - Cada actividad tiene un porcentaje de ejecución y una 'carga_por_dia' (peso).
   - El avance global se obtiene como la suma de (porcentaje_ejecutado * carga_por_dia) 
     dividida entre la suma total de todas las cargas.
   - De esta forma, las actividades que representan más trabajo diario (mayor carga)
     aportan proporcionalmente más al avance general.

2. Promedio simple de porcentajes:
   - Se suman todos los porcentajes de ejecución y se dividen entre el número de actividades.
   - Este cálculo asume que todas las actividades tienen la misma importancia, 
     independientemente de su carga real de trabajo.
   - Es incorrecto porque trata igual un metro de vereda (baja carga) que un kilómetro de carretera (alta carga),
     y podría dar un avance engañoso (por ejemplo, 100% de vereda + 0% de carretera → 50%).
   
Por eso, el método ponderado refleja fielmente el progreso real,
mientras que el promedio simple solo sirve para comparaciones o ilustrar el sesgo del cálculo no ponderado.
"""

"\nEn este script calculamos el avance mensual de dos formas:\n\n1. Avance ponderado por carga de trabajo:\n   - Cada actividad tiene un porcentaje de ejecución y una 'carga_por_dia' (peso).\n   - El avance global se obtiene como la suma de (porcentaje_ejecutado * carga_por_dia) \n     dividida entre la suma total de todas las cargas.\n   - De esta forma, las actividades que representan más trabajo diario (mayor carga)\n     aportan proporcionalmente más al avance general.\n\n2. Promedio simple de porcentajes:\n   - Se suman todos los porcentajes de ejecución y se dividen entre el número de actividades.\n   - Este cálculo asume que todas las actividades tienen la misma importancia, \n     independientemente de su carga real de trabajo.\n   - Es incorrecto porque trata igual un metro de vereda (baja carga) que un kilómetro de carretera (alta carga),\n     y podría dar un avance engañoso (por ejemplo, 100% de vereda + 0% de carretera → 50%).\n\nPor eso, el método ponderado refleja fielme

In [173]:
def calcular_promedio_avance_mal(porcentajes):
    """
    Devuelve el promedio simple de avance.
    - porcentajes: dict actividad -> porcentaje ejecutado (0..1)
    """
    total_actividades = len(porcentajes)
    if total_actividades == 0:
        return 0.0
    suma_porcentajes = sum(porcentajes.values())
    return suma_porcentajes / total_actividades

In [174]:
print(calcular_promedio_avance_mal(porcentajes=porcentajes_programado_vigente_mes)) # Este metodo esta mal, porque solo promedia los avances

0.027818114188272007


## Avance ejecutado en el mes

In [175]:
porcentajes_ejecutados_vigente_mes = {
    k: (cargas_trabajo_contratista_presente_mes.get(k, 0) / cargas_trabajo_programadas_totales[k]
        if cargas_trabajo_contratista_presente_mes.get(k, 0)  != 0 else 0)
    for k in cargas_trabajo_programadas_totales
}
print(porcentajes_ejecutados_vigente_mes)

{'MR101': 0.08365758754863814, 'MR102': 0, 'MR103': 0, 'MR104': 0, 'MR111': 0, 'MR112': 0, 'MR201': 0.08408037094281298, 'MR202': 0.17306028266512835, 'MR203': 0, 'MR204': 0, 'MR205': 0, 'MR206': 0, 'MR301': 0.22501406337896118, 'MR401': 0.07731560228854183, 'MR501': 0, 'MR601': 0.2499806216572359, 'MR701': 0, 'MR702': 0}


In [176]:
print(cargas_trabajo_contratista_presente_mes)

{'MR101': 0.86, 'MR202': 6.0, 'MR301': 3600.0, 'MR601': 64.5, 'MR201': 2720.0, 'MR401': 5}


In [177]:
porcentaje_ejecutado_vigente_mes=calcular_porcentaje(cargas_mes=cargas_trabajo_contratista_presente_mes,cargas_totales=cargas_trabajo_programadas_totales,carga_por_dia=carga_por_dia)
print(porcentaje_ejecutado_vigente_mes)

0.0591530651200461


In [178]:
data_to_see = {
    "porcentaje_programado_vigente_mes": porcentaje_programado_vigente_mes * 100,
    "porcentaje_ejecutado_vigente_mes": porcentaje_ejecutado_vigente_mes * 100,
    "porcentaje_programado_acumulado_vigente_mes": porcentaje_programado_vigente_mes
    * 100,
    "porcentaje_ejecutado_acumulado_vigente_mes": porcentaje_ejecutado_vigente_mes
    * 100,
}

In [179]:
data_to_print = {
    "porcentaje_programado_vigente_mes": porcentaje_programado_vigente_mes,
    "porcentaje_ejecutado_vigente_mes": porcentaje_ejecutado_vigente_mes,
    "porcentaje_programado_acumulado_vigente_mes": porcentaje_programado_vigente_mes,
    "porcentaje_ejecutado_acumulado_vigente_mes": porcentaje_ejecutado_vigente_mes,
}

In [180]:
ruta_directorio = os.path.join("output", proyecto, str(mes))
ruta_archivo = os.path.join(ruta_directorio, "cargas_trabajo.json")

# Crear el directorio si no existe
os.makedirs(ruta_directorio, exist_ok=True)
# Guardar el diccionario en un archivo JSON

with open(ruta_archivo, "w", encoding="utf-8") as archivo_json:
    json.dump(data_to_print, archivo_json, ensure_ascii=False, indent=4)
